# EDA 
For EDA analysis, we will use data from GCP

In [2]:
from google.cloud import bigquery

client = bigquery.Client(project='crmhetic')

In [8]:
query_account=""" select * from `crmhetic.raw_data.accounts` """
query_job = client.query(query_account)
results = query_job.result()
# to dataframe
df_account = results.to_dataframe()

query_products=""" select * from `crmhetic.raw_data.products` """
query_job = client.query(query_products)
results = query_job.result()
df_products = results.to_dataframe()


/Users/camille/repo/Hetic/repo_M2/eval_certifiante/etl/dbt_crm_venv/lib/python3.13/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [6]:
df

,account,sector,year_established,revenue,employees,office_location,subsidiary_of
0,Blackzim,retail,2009,497.11,1588,United States,None
1,Cancity,retail,2001,718.62,2448,United States,None
2,Fasehatice,retail,1990,4968.91,7523,United States,None
3,Ganjaflex,retail,1995,5158.71,17479,Japan,None
4,Gekko & Co,retail,1990,2520.83,3502,United States,None
...,...,...,...,...,...,...,...
80,Cheers,entertainment,1993,4269.90,6472,United States,Massive Dynamic
81,Codehow,software,1998,2714.90,2641,United States,Acme Corporation
82,Bluth Company,technolgy,1993,1242.32,3027,United States,Acme Corporation
83,Donquadtech,technolgy,1992,1712.68,3194,United States,Acme Corporation
